# Lab 2 - Automatic Creation of `sqlalchemy` data type `dict`

So far, we have been manually constructing the `sqlalchemy` type `dict`, but this approach quickly becomes unwieldy.  Let's look at using the `pandas` types to programmically construct the type `dict`.

In [1]:
import pandas as pd
from dfply import *
pd.__version__

'0.24.1'

In [2]:
!rm databases/baseball.db

## Case Study - People

Let's use the `People.csv` file from the [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/) as our motivating example, since 

1. It has examples of a number of types
2. It has lots of columns and would be annoying to manually construct the type `dict`.

In [3]:
people = pd.read_csv('/Users/bg8485bw/DSCI430/baseball/core/People.csv')
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [4]:
people.shape

(19370, 24)

## <font color="red"> Exercise 1 </font>

Inspect the types of the `people` table and make note of any necessary changes.

In [5]:
# Your code here
people.dtypes


playerID         object
birthYear       float64
birthMonth      float64
birthDay        float64
birthCountry     object
birthState       object
birthCity        object
deathYear       float64
deathMonth      float64
deathDay        float64
deathCountry     object
deathState       object
deathCity        object
nameFirst        object
nameLast         object
nameGiven        object
weight          float64
height          float64
bats             object
throws           object
debut            object
finalGame        object
retroID          object
bbrefID          object
dtype: object

*Your thoughts here*

debut and final game should be date time, 
the birth and month columns should be int64type.
Capital I ints can have missing values.

## Missing `Int64` columns

As mentioned in [Lecture 1/4](./pbpython/notebooks/1_4_more_on_pandas_data_types_key.ipynb), we need to use the most recent version `pandas` (still in development as of this writing) to allow us to have integer columns with missing values.

In [6]:
assert pd.__version__.startswith('0.24'), "Please uncomment and run the pip command to upgrade pandas"
#!pip install --upgrade --pre pandas

## Correcting the `pandas` types

1. We pass `parse_dates` a list of date columns
2. We pass `dtypes` a `dict` of types of the birth and death columns

#### Constructing the `dtype` `dict`

In [7]:
date_cols = ['debut', 'finalGame']

In [8]:
birth_death_date_cols = [prefix + time for prefix in ('birth', 'death') for time in ('Year', 'Month', 'Day')]
people_dtypes = {col:pd.Int64Dtype() for col in people.columns if col in birth_death_date_cols}
people_dtypes

{'birthDay': Int64Dtype(),
 'birthMonth': Int64Dtype(),
 'birthYear': Int64Dtype(),
 'deathDay': Int64Dtype(),
 'deathMonth': Int64Dtype(),
 'deathYear': Int64Dtype()}

## Rereading the csv with the correct types

In [9]:
people = pd.read_csv('/Users/bg8485bw/DSCI430/baseball/core/People.csv', dtype=people_dtypes, parse_dates=date_cols)
people.dtypes

playerID                object
birthYear                Int64
birthMonth               Int64
birthDay                 Int64
birthCountry            object
birthState              object
birthCity               object
deathYear                Int64
deathMonth               Int64
deathDay                 Int64
deathCountry            object
deathState              object
deathCity               object
nameFirst               object
nameLast                object
nameGiven               object
weight                 float64
height                 float64
bats                    object
throws                  object
debut           datetime64[ns]
finalGame       datetime64[ns]
retroID                 object
bbrefID                 object
dtype: object

## <font color="red"> Exercise 2 </font>
**Goal:** Find a method/attribute of each `dtype`, which preferably returns something immutable like a `str`, that we can use to identify the general type.
**Tasks:**

1. Pull off an example `dtype`
2. Use `dir` to inspect the available methods
3. Test the methods/attributes to find a good candidate.

In [11]:
# Your code here
a = first(people).dtype
[m for m in dir(a) if not m.startswith("__")]

['alignment',
 'base',
 'byteorder',
 'char',
 'descr',
 'fields',
 'flags',
 'hasobject',
 'isalignedstruct',
 'isbuiltin',
 'isnative',
 'itemsize',
 'kind',
 'metadata',
 'name',
 'names',
 'ndim',
 'newbyteorder',
 'num',
 'shape',
 'str',
 'subdtype',
 'type']

In [12]:
# name is a good candidate but uses to much memory, kind is better
[t.kind for t in people.dtypes]

['O',
 'i',
 'i',
 'i',
 'O',
 'O',
 'O',
 'i',
 'i',
 'i',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'f',
 'f',
 'O',
 'O',
 'M',
 'M',
 'O',
 'O']

In [13]:
people.dtypes

playerID                object
birthYear                Int64
birthMonth               Int64
birthDay                 Int64
birthCountry            object
birthState              object
birthCity               object
deathYear                Int64
deathMonth               Int64
deathDay                 Int64
deathCountry            object
deathState              object
deathCity               object
nameFirst               object
nameLast                object
nameGiven               object
weight                 float64
height                 float64
bats                    object
throws                  object
debut           datetime64[ns]
finalGame       datetime64[ns]
retroID                 object
bbrefID                 object
dtype: object

## Creating a type conversion dictionary

1. keys will be the `dtype.kind` strings
2. The `values` will be the associated `sqlalchemy` types

In [14]:
from sqlalchemy import Integer, Float, String, DateTime
DTYPES_TO_SQLALCHEMY_TYPES = {'O':String,
                              'i':Integer,
                              'f':Float,
                              'M':DateTime}
DTYPES_TO_SQLALCHEMY_TYPES ## <---- Treat this as a constant fix for dtypes.

{'M': sqlalchemy.sql.sqltypes.DateTime,
 'O': sqlalchemy.sql.sqltypes.String,
 'f': sqlalchemy.sql.sqltypes.Float,
 'i': sqlalchemy.sql.sqltypes.Integer}

## Use ALL CAPS for global constants

When dealing with global constants, we should

1. Define them at the top of the file.
2. Use an ALL CAPS name to make them stand out.

In [15]:
[(col, people[col].is_unique) for col in people]

[('playerID', True),
 ('birthYear', False),
 ('birthMonth', False),
 ('birthDay', False),
 ('birthCountry', False),
 ('birthState', False),
 ('birthCity', False),
 ('deathYear', False),
 ('deathMonth', False),
 ('deathDay', False),
 ('deathCountry', False),
 ('deathState', False),
 ('deathCity', False),
 ('nameFirst', False),
 ('nameLast', False),
 ('nameGiven', False),
 ('weight', False),
 ('height', False),
 ('bats', False),
 ('throws', False),
 ('debut', False),
 ('finalGame', False),
 ('retroID', False),
 ('bbrefID', False)]

## <font color="red"> Exercise 3 </font>

Write a `dict` comprehension that uses our conversion `dict` to convert the `pandas` `dtypes` to `sqlalchemy` types.

In [16]:
# Your code here
from toolz import get
{col:get(dtype.kind, DTYPES_TO_SQLALCHEMY_TYPES) 
 for (col,dtype) in zip(people.columns,people.dtypes)}

{'bats': sqlalchemy.sql.sqltypes.String,
 'bbrefID': sqlalchemy.sql.sqltypes.String,
 'birthCity': sqlalchemy.sql.sqltypes.String,
 'birthCountry': sqlalchemy.sql.sqltypes.String,
 'birthDay': sqlalchemy.sql.sqltypes.Integer,
 'birthMonth': sqlalchemy.sql.sqltypes.Integer,
 'birthState': sqlalchemy.sql.sqltypes.String,
 'birthYear': sqlalchemy.sql.sqltypes.Integer,
 'deathCity': sqlalchemy.sql.sqltypes.String,
 'deathCountry': sqlalchemy.sql.sqltypes.String,
 'deathDay': sqlalchemy.sql.sqltypes.Integer,
 'deathMonth': sqlalchemy.sql.sqltypes.Integer,
 'deathState': sqlalchemy.sql.sqltypes.String,
 'deathYear': sqlalchemy.sql.sqltypes.Integer,
 'debut': sqlalchemy.sql.sqltypes.DateTime,
 'finalGame': sqlalchemy.sql.sqltypes.DateTime,
 'height': sqlalchemy.sql.sqltypes.Float,
 'nameFirst': sqlalchemy.sql.sqltypes.String,
 'nameGiven': sqlalchemy.sql.sqltypes.String,
 'nameLast': sqlalchemy.sql.sqltypes.String,
 'playerID': sqlalchemy.sql.sqltypes.String,
 'retroID': sqlalchemy.sql.sqltyp

## <font color="red"> Exercise 4 </font>

Package your expression in a `lambda` and refactor your code by adding helper functions to clean up the expression.

In [17]:
# lambda function before refactoring
def get_sql_types(df):
    sql_type = lambda type: get(type.kind,DTYPES_TO_SQLALCHEMY_TYPES)
    cols_and_dtypes = lambda df: zip(df.columns,df.dtypes)
    return {cols:sql_type(dtypes) for cols,dtypes in cols_and_dtypes(df)}


In [18]:
# refactored lambda function and helper functions
get_sql_types(people)

{'bats': sqlalchemy.sql.sqltypes.String,
 'bbrefID': sqlalchemy.sql.sqltypes.String,
 'birthCity': sqlalchemy.sql.sqltypes.String,
 'birthCountry': sqlalchemy.sql.sqltypes.String,
 'birthDay': sqlalchemy.sql.sqltypes.Integer,
 'birthMonth': sqlalchemy.sql.sqltypes.Integer,
 'birthState': sqlalchemy.sql.sqltypes.String,
 'birthYear': sqlalchemy.sql.sqltypes.Integer,
 'deathCity': sqlalchemy.sql.sqltypes.String,
 'deathCountry': sqlalchemy.sql.sqltypes.String,
 'deathDay': sqlalchemy.sql.sqltypes.Integer,
 'deathMonth': sqlalchemy.sql.sqltypes.Integer,
 'deathState': sqlalchemy.sql.sqltypes.String,
 'deathYear': sqlalchemy.sql.sqltypes.Integer,
 'debut': sqlalchemy.sql.sqltypes.DateTime,
 'finalGame': sqlalchemy.sql.sqltypes.DateTime,
 'height': sqlalchemy.sql.sqltypes.Float,
 'nameFirst': sqlalchemy.sql.sqltypes.String,
 'nameGiven': sqlalchemy.sql.sqltypes.String,
 'nameLast': sqlalchemy.sql.sqltypes.String,
 'playerID': sqlalchemy.sql.sqltypes.String,
 'retroID': sqlalchemy.sql.sqltyp

## <font color="red"> Exercise 5 </font>

Add the `People.csv` to your `baseball.db`

In [19]:
# Fire up the engines!
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select
from sqlalchemy import create_engine

mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [20]:
schema = pd.io.sql.get_schema(people, 'people', keys='playerID', con=mang_eng, dtype=get_sql_types(people))
print(schema)


CREATE TABLE people (
	"playerID" VARCHAR NOT NULL, 
	"birthYear" INTEGER, 
	"birthMonth" INTEGER, 
	"birthDay" INTEGER, 
	"birthCountry" VARCHAR, 
	"birthState" VARCHAR, 
	"birthCity" VARCHAR, 
	"deathYear" INTEGER, 
	"deathMonth" INTEGER, 
	"deathDay" INTEGER, 
	"deathCountry" VARCHAR, 
	"deathState" VARCHAR, 
	"deathCity" VARCHAR, 
	"nameFirst" VARCHAR, 
	"nameLast" VARCHAR, 
	"nameGiven" VARCHAR, 
	weight FLOAT, 
	height FLOAT, 
	bats VARCHAR, 
	throws VARCHAR, 
	debut DATETIME, 
	"finalGame" DATETIME, 
	"retroID" VARCHAR, 
	"bbrefID" VARCHAR, 
	CONSTRAINT people_pk PRIMARY KEY ("playerID")
)




In [21]:
mang_eng.execute(schema)

2019-02-08 13:22:23,620 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-08 13:22:23,623 INFO sqlalchemy.engine.base.Engine ()
2019-02-08 13:22:23,627 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-08 13:22:23,629 INFO sqlalchemy.engine.base.Engine ()
2019-02-08 13:22:23,633 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE people (
	"playerID" VARCHAR NOT NULL, 
	"birthYear" INTEGER, 
	"birthMonth" INTEGER, 
	"birthDay" INTEGER, 
	"birthCountry" VARCHAR, 
	"birthState" VARCHAR, 
	"birthCity" VARCHAR, 
	"deathYear" INTEGER, 
	"deathMonth" INTEGER, 
	"deathDay" INTEGER, 
	"deathCountry" VARCHAR, 
	"deathState" VARCHAR, 
	"deathCity" VARCHAR, 
	"nameFirst" VARCHAR, 
	"nameLast" VARCHAR, 
	"nameGiven" VARCHAR, 
	weight FLOAT, 
	height FLOAT, 
	bats VARCHAR, 
	throws VARCHAR, 
	debut DATETIME, 
	"finalGame" DATETIME, 
	"retroID" VARCHAR, 
	"bbrefID" VARCHAR, 
	CONSTRAINT people_pk P

In [22]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
People = Base.classes.people

In [23]:
people.to_sql('people', 
                con=mang_eng, 
                dtype=get_sql_types(people), 
                index=False,
                if_exists='append')

2019-02-08 13:22:24,839 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("people")
2019-02-08 13:22:24,843 INFO sqlalchemy.engine.base.Engine ()
2019-02-08 13:22:24,880 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-08 13:22:25,530 INFO sqlalchemy.engine.base.Engine INSERT INTO people ("playerID", "birthYear", "birthMonth", "birthDay", "birthCountry", "birthState", "birthCity", "deathYear", "deathMonth", "deathDay", "deathCountry", "deathState", "deathCity", "nameFirst", "nameLast", "nameGiven", weight, height, bats, throws, debut, "finalGame", "retroID", "bbrefID") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-02-08 13:22:25,532 INFO sqlalchemy.engine.base.Engine (('aardsda01', 1981, 12, 27, 'USA', 'CO', 'Denver', None, None, None, None, None, None, 'David', 'Aardsma', 'David Allan', 215.0, 75.0, 'R', 'R', '2004-04-06 00:00:00.000000', '2015-08-23 00:00:00.000000', 'aardd001', 'aardsda01'), ('aaronha01', 1934, 2, 5, 'USA', 'AL', 

In [24]:
# Looks good
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(People)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-02-08 13:22:25,844 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-08 13:22:25,846 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM people
2019-02-08 13:22:25,851 INFO sqlalchemy.engine.base.Engine ()


[{'bats': 'R',
  'bbrefID': 'aardsda01',
  'birthCity': 'Denver',
  'birthCountry': 'USA',
  'birthDay': 27,
  'birthMonth': 12,
  'birthState': 'CO',
  'birthYear': 1981,
  'deathCity': None,
  'deathCountry': None,
  'deathDay': None,
  'deathMonth': None,
  'deathState': None,
  'deathYear': None,
  'debut': '2004-04-06 00:00:00.000000',
  'finalGame': '2015-08-23 00:00:00.000000',
  'height': 75.0,
  'nameFirst': 'David',
  'nameGiven': 'David Allan',
  'nameLast': 'Aardsma',
  'playerID': 'aardsda01',
  'retroID': 'aardd001',
  'throws': 'R',
  'weight': 215.0},
 {'bats': 'R',
  'bbrefID': 'aaronha01',
  'birthCity': 'Mobile',
  'birthCountry': 'USA',
  'birthDay': 5,
  'birthMonth': 2,
  'birthState': 'AL',
  'birthYear': 1934,
  'deathCity': None,
  'deathCountry': None,
  'deathDay': None,
  'deathMonth': None,
  'deathState': None,
  'deathYear': None,
  'debut': '1954-04-13 00:00:00.000000',
  'finalGame': '1976-10-03 00:00:00.000000',
  'height': 72.0,
  'nameFirst': 'Hank',

## <font color="red"> Exercise 6 </font>

Set up a similar automatic conversion precess for `pyspark.DataFrames`. You need to build a structure like this:

```python
schema = StructType([StructField('Name', StringType(), True),
                     StructField('DateTime', TimestampType(), True)
                     StructField('Age', IntegerType(), True)])
```

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean
from pyspark.sql.types import *

spark1 = SparkSession.builder.appName('Ops').getOrCreate()
df_spark = spark1.read.csv('/Users/bg8485bw/DSCI430/baseball/core/People.csv', inferSchema=True, header=True)

In [29]:
[t.kind for t in people.dtypes]

['O',
 'i',
 'i',
 'i',
 'O',
 'O',
 'O',
 'i',
 'i',
 'i',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'f',
 'f',
 'O',
 'O',
 'M',
 'M',
 'O',
 'O']

In [30]:
DTYPES_TO_PYSPARK_TYPES = {'O':StringType,
                              'i':IntegerType,
                              'f':FloatType,
                              'M':TimestampType}

In [31]:
# List form instead of dict
from toolz import get
from itertools import *
[(col,get(dtype.kind, DTYPES_TO_PYSPARK_TYPES)(),True)
 for (col,dtype) in zip(people.columns,people.dtypes)]

[('playerID', StringType, True),
 ('birthYear', IntegerType, True),
 ('birthMonth', IntegerType, True),
 ('birthDay', IntegerType, True),
 ('birthCountry', StringType, True),
 ('birthState', StringType, True),
 ('birthCity', StringType, True),
 ('deathYear', IntegerType, True),
 ('deathMonth', IntegerType, True),
 ('deathDay', IntegerType, True),
 ('deathCountry', StringType, True),
 ('deathState', StringType, True),
 ('deathCity', StringType, True),
 ('nameFirst', StringType, True),
 ('nameLast', StringType, True),
 ('nameGiven', StringType, True),
 ('weight', FloatType, True),
 ('height', FloatType, True),
 ('bats', StringType, True),
 ('throws', StringType, True),
 ('debut', TimestampType, True),
 ('finalGame', TimestampType, True),
 ('retroID', StringType, True),
 ('bbrefID', StringType, True)]

In [41]:
# Package into lambdas
def get_pyspark_types(df):
    t = True
    spark_types = lambda type:get(type.kind, DTYPES_TO_PYSPARK_TYPES)()
    cols_and_dtpyes = lambda df:zip(df.columns, df.dtypes)
    return [(cols,spark_types(dtypes),t) for (cols, dtypes) in cols_and_dtpyes(df)]
    
   

In [42]:
# Test function
# Not sure if it only can be instantiated once?
get_pyspark_types(people)

AttributeError: 'tuple' object has no attribute 'kind'

In [49]:
# Seems to be the case
sf = [StructField(names,type,null) for (names,type,null) in get_pyspark_types(people)]
StructType(sf)

AttributeError: 'tuple' object has no attribute 'kind'

In [50]:
# Assign StructType to StructFields
schema1 = StructType(sf)

In [51]:
# Read in the data into spark
people = spark1.read.csv('/Users/bg8485bw/DSCI430/baseball/core/People.csv', header=True, schema=schema1, nullValue='-')
people

DataFrame[playerID: string, birthYear: int, birthMonth: int, birthDay: int, birthCountry: string, birthState: string, birthCity: string, deathYear: int, deathMonth: int, deathDay: int, deathCountry: string, deathState: string, deathCity: string, nameFirst: string, nameLast: string, nameGiven: string, weight: float, height: float, bats: string, throws: string, debut: timestamp, finalGame: timestamp, retroID: string, bbrefID: string]

In [52]:
# Print schema
people.printSchema()

root
 |-- playerID: string (nullable = true)
 |-- birthYear: integer (nullable = true)
 |-- birthMonth: integer (nullable = true)
 |-- birthDay: integer (nullable = true)
 |-- birthCountry: string (nullable = true)
 |-- birthState: string (nullable = true)
 |-- birthCity: string (nullable = true)
 |-- deathYear: integer (nullable = true)
 |-- deathMonth: integer (nullable = true)
 |-- deathDay: integer (nullable = true)
 |-- deathCountry: string (nullable = true)
 |-- deathState: string (nullable = true)
 |-- deathCity: string (nullable = true)
 |-- nameFirst: string (nullable = true)
 |-- nameLast: string (nullable = true)
 |-- nameGiven: string (nullable = true)
 |-- weight: float (nullable = true)
 |-- height: float (nullable = true)
 |-- bats: string (nullable = true)
 |-- throws: string (nullable = true)
 |-- debut: timestamp (nullable = true)
 |-- finalGame: timestamp (nullable = true)
 |-- retroID: string (nullable = true)
 |-- bbrefID: string (nullable = true)



In [54]:
# Test head
# Works!
people.take(5)

[Row(playerID='aardsda01', birthYear=1981, birthMonth=12, birthDay=27, birthCountry='USA', birthState='CO', birthCity='Denver', deathYear=None, deathMonth=None, deathDay=None, deathCountry=None, deathState=None, deathCity=None, nameFirst='David', nameLast='Aardsma', nameGiven='David Allan', weight=215.0, height=75.0, bats='R', throws='R', debut=datetime.datetime(2004, 4, 6, 0, 0), finalGame=datetime.datetime(2015, 8, 23, 0, 0), retroID='aardd001', bbrefID='aardsda01'),
 Row(playerID='aaronha01', birthYear=1934, birthMonth=2, birthDay=5, birthCountry='USA', birthState='AL', birthCity='Mobile', deathYear=None, deathMonth=None, deathDay=None, deathCountry=None, deathState=None, deathCity=None, nameFirst='Hank', nameLast='Aaron', nameGiven='Henry Louis', weight=180.0, height=72.0, bats='R', throws='R', debut=datetime.datetime(1954, 4, 13, 0, 0), finalGame=datetime.datetime(1976, 10, 3, 0, 0), retroID='aaroh101', bbrefID='aaronha01'),
 Row(playerID='aaronto01', birthYear=1939, birthMonth=8,